# Sobre o dataset

O conjunto de dados Iris, um conjunto de dados bem conhecido no campo de aprendizado de máquina, foi introduzido no artigo de 1936 de R.A. Fisher, "The Use of Multiple Measurements in Taxonomic Problems" (O Uso de Múltiplas Medidas em Problemas Taxonômicos). Ele contém medições de três espécies de flores de íris: Setosa, Versicolor e Virginica. Cada espécie possui 50 amostras, totalizando 150 amostras no conjunto de dados. O conjunto de dados inclui as seguintes propriedades para cada flor:

    .Id: Um identificador único
    .SepalLengthCm: Comprimento da sépala em centímetros
    .SepalWidthCm: Largura da sépala em centímetros
    .PetalLengthCm: Comprimento da pétala em centímetros
    .PetalWidthCm: Largura da pétala em centímetros
    .Species: Espécie da flor

# Importando dataset e verifcando a integridade dos dados

In [27]:
import pandas as pd

pd.set_option('display.max_columns', 30)
df = pd.read_csv('../datasets/iris.csv')

df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             150 non-null    int64  
 1   SepalLengthCm  150 non-null    float64
 2   SepalWidthCm   150 non-null    float64
 3   PetalLengthCm  150 non-null    float64
 4   PetalWidthCm   150 non-null    float64
 5   Species        150 non-null    object 
dtypes: float64(4), int64(1), object(1)
memory usage: 7.2+ KB


In [29]:
df.describe()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,75.500000,5.843333,3.054000,3.758667,1.198667
std,43.445368,0.828066,0.433594,1.764420,0.763161
min,1.000000,4.300000,2.000000,1.000000,0.100000
25%,38.250000,5.100000,2.800000,1.600000,0.300000
50%,75.500000,5.800000,3.000000,4.350000,1.300000
75%,112.750000,6.400000,3.300000,5.100000,1.800000
max,150.000000,7.900000,4.400000,6.900000,2.500000


In [30]:
df['Species'].value_counts()

Species
Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
Name: count, dtype: int64

# Pré-processamento dos dados

Nesta etapa iremos realizar as seguintes tarefas:

    1. Drop de dados sem relevancia (id) e divisao do dataset em duas variaveis x e y
    2. Converção dos dados qualitativos em quantitativos discretos para a possibilidade de classificação do modelo
    3. Divisão dos dados de treino e teste

In [31]:
x = df.drop(columns=['Id', 'Species'])

In [32]:
from sklearn.preprocessing import LabelEncoder

labelEncoder = LabelEncoder()

y = labelEncoder.fit_transform(df['Species'])

In [33]:
from sklearn.model_selection import train_test_split

x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, stratify=y, test_size=0.3)

Verificando o balanceamento de classes

# Construção e treinamento do modelo

Dividiremos este processo em duas etapas, sendo a primeira etapa para descobrir os melhores parametros para contruir nosso modelo de KNN e outra para treinar um novo modelo utilizando esses parametros chave para ter um modelo mais eficiente

1° Descobrindo os hiperparametros utilizando a função GridSearchCV

A função <a href='https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html'>GridSearchCV</a> é uma função da biblioteca Scikit-learn usada para encontrar os melhores parâmetros para o nosso modelo. Ela funciona testando todas as possiveis combinações de valores de parâmetros, treinando o modelo várias vezes para cada combinação. Ao final, ela identifica e retorna a melhor combinação de parâmetros, aquela que produz o melhor desempenho do modelo. Essa ferramenta é extremamente útil quando queremos melhorar a performance do modelo ajustando seus parâmetros de forma precisa.

In [34]:
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

#Normalizando variaveis preditoras
normalizador = MinMaxScaler(feature_range=(0,1))
X_norm = normalizador.fit_transform(x)

#Deifinindo valores que serão testados no KNN:
valores_K = np.array([3,5,7,9,11])
calculo_distancia=['minkowski', 'chebyshev']
valores_p = np.array([1,2,3,4])
valores_grid = {'n_neighbors':valores_K, 'metric':calculo_distancia, 'p':valores_p}


#Criação do modelo:
modelo = KNeighborsClassifier()
gridKNN = GridSearchCV(estimator=modelo, param_grid=valores_grid)
gridKNN.fit(X_norm, y)

#Score:
print(f'Melhor Acuracia: {gridKNN.best_score_}')
print(f'Melhor K: {gridKNN.best_estimator_.n_neighbors}')
print(f'Melhor Distancia: {gridKNN.best_estimator_.metric}')
print(f'Melhor valor p: {gridKNN.best_estimator_.p}')

Melhor Acuracia: 0.9733333333333334
Melhor K: 5
Melhor Distancia: minkowski
Melhor valor p: 3


2° Criando e treinando nosso modelo

In [35]:
from sklearn.neighbors import KNeighborsClassifier

#Criação do modelo:
modelo = KNeighborsClassifier(n_neighbors=5, 
                              weights='uniform', 
                              algorithm='auto', 
                              leaf_size=30, 
                              p=3, 
                              metric='minkowski', 
                              metric_params=None, 
                              n_jobs=None)
modelo.fit(x_treino, y_treino)

#Score:
resultado = modelo.score(x_teste, y_teste)
print(f'Acuracia: {resultado}')

Acuracia: 0.9777777777777777
